In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.model_selection import train_test_split, KFold
from skrebate import ReliefF  # Ensure you have skrebate installed for ReliefF
import xgboost as xgb
from scipy.spatial.distance import jensenshannon

# Load the dataset
file_path = "PCOS_data_without_infertility.xlsx"
df = pd.read_excel(file_path, sheet_name="Full_new")

# Data Preprocessing
df = df.drop(columns=['Sl. No', 'Patient File No.', 'Unnamed: 44'])
df = df.apply(pd.to_numeric, errors='coerce')
df.fillna(df.median(), inplace=True)

# Encode categorical columns
le = LabelEncoder()
categorical_columns = ['Blood Group', 'Cycle(R/I)', 'Pregnant(Y/N)', 
                       'Weight gain(Y/N)', 'hair growth(Y/N)', 
                       'Skin darkening (Y/N)', 'Hair loss(Y/N)', 
                       'Pimples(Y/N)', 'Fast food (Y/N)', 
                       'Reg.Exercise(Y/N)']
for col in categorical_columns:
    if col in df.columns:
        df[col] = le.fit_transform(df[col])

# Split into features and target
X = df.drop(columns=['PCOS (Y/N)'])
y = df['PCOS (Y/N)']

# Jensen-Shannon Divergence based stability calculation
def calculate_jsd_stability(X, y, selected_features):
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    fold_rankings = []

    for train_idx, test_idx in kf.split(X):
        X_train_fold, X_test_fold = X.iloc[train_idx], X.iloc[test_idx]
        y_train_fold, y_test_fold = y.iloc[train_idx], y.iloc[test_idx]

        model = xgb.XGBClassifier(eval_metric='mlogloss')
        model.fit(X_train_fold[selected_features], y_train_fold)
        
        # Get feature importances and normalize them as a probability distribution
        feature_importances = model.feature_importances_
        feature_importances /= np.sum(feature_importances)  # Normalize to sum 1
        fold_rankings.append(feature_importances)

    fold_rankings = np.array(fold_rankings)
    stability_scores = []

    # Compute JSD between all pairs of fold rankings
    for i in range(len(fold_rankings)):
        for j in range(i + 1, len(fold_rankings)):
            jsd = jensenshannon(fold_rankings[i], fold_rankings[j])
            stability_scores.append(jsd)

    # Return the average JSD stability score (1 - mean JSD, as lower JSD indicates higher similarity)
    return 1 - np.mean(stability_scores)

# Thresholds to evaluate
thresholds = [17, 23, 28, 33]

# Feature Selection Methods and Stability Calculation
for threshold in thresholds:
    print(f"\nThreshold: {threshold}")
    # Chi-Square
    chi_selector = SelectKBest(chi2, k=threshold)
    chi_selector.fit(X, y)
    chi_features = X.columns[chi_selector.get_support()]
    chi_stability = calculate_jsd_stability(X, y, chi_features)
    print(f"Chi-Square Stability: {chi_stability}")

    # Pearson Correlation
    corr_matrix = df.corr()
    top_corr_features = corr_matrix.index[abs(corr_matrix['PCOS (Y/N)']) > 0.2]
    
    # Select features based on threshold and ensure correct indexing
    pearson_features_mask = X.columns.isin(top_corr_features)
    
    # Get the top features based on correlation with PCOS (Y/N)
    pearson_feature_names = X.columns[pearson_features_mask][:threshold]
    
    pearson_stability = calculate_jsd_stability(X, y, pearson_feature_names)
    print(f"Pearson Stability: {pearson_stability}")

    # ReliefF
    relief_selector = ReliefF(n_neighbors=10)
    relief_selector.fit(X.values, y.values)
    
    # Get indices of top features based on importance scores
    relief_features_indices = np.argsort(relief_selector.feature_importances_)[-threshold:]
    
    # Select feature names based on indices
    relief_feature_names = X.columns[relief_features_indices]
    
    relief_stability = calculate_jsd_stability(X, y, relief_feature_names)
    print(f"ReliefF Stability: {relief_stability}")

    # Mutual Information
    mutual_info_selector = SelectKBest(mutual_info_classif, k=threshold)
    mutual_info_selector.fit(X, y)
    
    # Get selected features from mutual information
    mutual_info_features = X.columns[mutual_info_selector.get_support()]
    
    mutual_info_stability = calculate_jsd_stability(X, y, mutual_info_features)
    print(f"Mutual Information Stability: {mutual_info_stability}")

    # Improved Ensemble: Weighted Voting of Features
    all_selected_features = list(chi_features) + list(pearson_feature_names) + list(relief_feature_names) + list(mutual_info_features)
    feature_weights = pd.Series(all_selected_features).value_counts()  # Count occurrences of each feature
    
    # Select top features based on their weights (occurrences in different methods)
    ensemble_selected_features = feature_weights[feature_weights > 1].index[:threshold]  # Select only those appearing in more than 1 method
    
    print(f"Total ensemble feature count for threshold {threshold}: {len(ensemble_selected_features)}")
    print(f"Ensemble Features: {ensemble_selected_features}")
    ensemble_stability = calculate_jsd_stability(X, y, list(ensemble_selected_features))
    print(f"Improved Ensemble Stability: {ensemble_stability}")



Threshold: 17
Chi-Square Stability: 0.9100385770180721
Pearson Stability: 0.9190543881758789
ReliefF Stability: 0.9152386402625874
Mutual Information Stability: 0.9081465321252543
Total ensemble feature count for threshold 17: 14
Ensemble Features: Index(['Fast food (Y/N)', 'Weight gain(Y/N)', 'Follicle No. (R)',
       'Follicle No. (L)', 'Cycle(R/I)', 'Pimples(Y/N)',
       'Skin darkening (Y/N)', 'AMH(ng/mL)', 'hair growth(Y/N)', 'Weight (Kg)',
       'FSH/LH', 'Cycle length(days)', '  I   beta-HCG(mIU/mL)',
       'Waist:Hip Ratio'],
      dtype='object')
Improved Ensemble Stability: 0.9216980158277719

Threshold: 23
Chi-Square Stability: 0.8936632185486372
Pearson Stability: 0.9190543881758789
ReliefF Stability: 0.8842702508570819
Mutual Information Stability: 0.8817626916312548
Total ensemble feature count for threshold 23: 22
Ensemble Features: Index(['Pimples(Y/N)', 'Follicle No. (R)', 'hair growth(Y/N)',
       'Weight gain(Y/N)', 'Fast food (Y/N)', 'AMH(ng/mL)', 'Follicle No